In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.proxy import Proxy, ProxyType
import requests
from dotenv import load_dotenv
from os import environ
import random
import pandas as pd
import time
from fake_useragent import UserAgent

In [ ]:
load_dotenv()
random.seed()

In [ ]:
path = environ["WEBDRIVER_PATH"]
url = environ["BASE_URL"]
feed_endpoint = environ["FEED_ENDPOINT"]
login_uid_xpath = '//*[@id="session_key"]' 
login_password_xpath = '//*[@id="session_password"]'
login_submit_xpath = '//*[@id="main-content"]/section[1]/div/div/form/button'

In [ ]:
phone_skip_btn = "/html/body/div/div[1]/section/div[2]/div/article/footer/div/div/button"

In [ ]:
# options = Options()
# ua = UserAgent()
# user_agent = ua.random

# options.add_argument(f'user-agent={user_agent}')
driver = webdriver.Chrome(
        # options=options, 
        executable_path=path
    )
driver.get(url)
uid_field = driver.find_element(By.XPATH, login_uid_xpath)
uid_field.send_keys(environ["LINKEDIN_USERNAME"])
password_field = driver.find_element(By.XPATH, login_password_xpath)
password_field.send_keys(environ["LINKEDIN_PASSWORD"])
okay_btn = driver.find_element(By.XPATH, login_submit_xpath).click()
if environ["ADD_PHONE_ENDPOINT"] in driver.current_url:
    btn = driver.find_element(By.XPATH, phone_skip_btn)
    btn.click()
    btn = None
time.sleep(5)
driver.quit()

In [ ]:
with open("../data_store/uids.csv", "rt", encoding="utf-8") as data:
    uid_list = data.read().split("\n")
uid_list.sort()
print(uid_list)

## Traversing JSON feed data for own profile

We will now, parse the json file and take only the necessary details.

In [ ]:
import json

file_path = "/home/prithoo/Coding/B2B_ProofOfConcept/output_data/search_results/prithoo11335@gmail.com_feed.json"

with open(file_path)as json_file:
    feed = json.load(json_file)

feed = feed["feed"]

feed_list = []
for post in feed:
    if "position" in post.get("content"):
        data_dict = {}
        data_dict["author"] = post.get("author_profile", "")
        data_dict["post"] = post.get("content", "")
        data_dict["urn"] = post.get("url", "").split("/")[-1]
        data_dict["link"] = post.get("url", "").lower()
        feed_list.append(data_dict)

# print(feed_list)

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(feed_list)
df.head()
op_file_name = f"/home/prithoo/Coding/B2B_ProofOfConcept/cleaned_data/{file_path.split('/')[-1]}.csv"
df.to_csv(op_file_name)

In [ ]:
import json

json_file_path = "/home/prithoo/Coding/B2B_ProofOfConcept/output_data/profiles/prithoo11335.json"

with open(json_file_path)as json_file:
    uids_dict_list = json.load(json_file).get("connections", [])


In [ ]:
main_list = []
for item in uids_dict_list:
    main_list.append(item.get("public_id", ""))

main_list = list(set(main_list))
main_list.sort()

In [ ]:
for item in main_list:
    with open("/home/prithoo/Coding/B2B_ProofOfConcept/data_store/universal_identifiers.csv", "a+t", encoding="utf-8")as list_file:
        list_file.write(f"{item}\n")


In [ ]:
from apis.api import LinkedInAPI

main_api = LinkedInAPI()

main_api.get_jobs("python")

In [11]:
import json 

file_path = "/home/prithoo/Coding/B2B_ProofOfConcept/raw_data/search_results/posts/python_content.json"
# file_path = "/home/prithoo/Coding/B2B_ProofOfConcept/raw_data/search_results/posts/python_all.json"
with open(file_path, "r+t", encoding="utf-8")as json_file:
    data = json.load(json_file)


In [12]:
type_list = []
for item in data:
    type_list.append(item.get("type", ""))

type_list = list(set(type_list))
type_list

['PROFILE', 'COMPANY', 'GROUP']

In [ ]:
import pandas as pd

word_list_file = "/home/prithoo/Documents/datasets/english/words.csv"

df = pd.read_csv(word_list_file, encoding="utf-8", header=1, names=["word"])

In [ ]:
df = df.filter(["word"], axis=1)
w_list = df["word"].tolist()
w_list

In [ ]:
w_list = [str(w) for w in w_list]
cleaned_list = [w.lower() for w in w_list if w.isalpha()]


In [ ]:
for item in cleaned_list:
    with open("/home/prithoo/Documents/datasets/english/cleaned_word_list.csv", "a+t", encoding="utf-8")as list_file:
        list_file.write(f"{item}\n")

In [ ]:
import requests

black_list = ["light", "hammer", "novice", "quintessential", "huerhvo", "cuwchwvuv", "chbvervkrbv"]
clean_words = []

cont = []
for item in black_list:
    print(f"Testing {item}")
    url = f"https://api.datamuse.com/words?ml={item}"
    r = requests.get(url)
    print(r.json())
    if r.json().get() > 95_000:
        clean_words.append(item)



In [ ]:
cont